In [22]:
import pandas as pd
import numpy as np
from transformers import *

import matplotlib.pyplot as plt

In [23]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [24]:
train.dropna(inplace=True)

In [25]:
total_missing = train.isnull().sum().sort_values(ascending = False)
total_missing

sentiment        0
selected_text    0
text             0
textID           0
dtype: int64

In [29]:
train.iloc[20709]

textID                                                  6109fbeca5
text             Watching Almost Famous with Andrew, Michael, a...
selected_text    Watching Almost Famous with Andrew, Michael, a...
sentiment                                                  neutral
Name: 20710, dtype: object

In [28]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [7]:
train.groupby('sentiment')['textID'].nunique()

sentiment
negative     7781
neutral     11117
positive     8582
Name: textID, dtype: int64

In [18]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize(text, sentiment):
    output = tokenizer.encode_plus(sentiment, text, add_special_tokens=True)
    return output['input_ids']

train['new_column1'] = train.apply(lambda x: tokenize(x['text'], x['sentiment']), axis=1)

print('max tokenized_text size: ', max(len(s) for s in train['new_column1']))

max tokenized_text size:  105


[0,
 33407,
 2,
 2,
 407,
 3036,
 208,
 2606,
 38,
 40,
 2649,
 47,
 259,
 11,
 764,
 3402,
 16506,
 2]

In [19]:
word2idx = []
for tokenizer in tokenizers:
    word2idx.append(tokenizer.word_to_tokens)

Using bos_token, but it is not set yet.


In [44]:
import flair

In [53]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
s = flair.data.Word('you')
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment

2020-07-04 20:06:52,843 loading file /Users/pattersonwu/.flair/models/sentiment-en-mix-distillbert.pt


AttributeError: module 'flair.data' has no attribute 'Word'